In [2]:
import os
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
RootPath = r"D:\Hagar\Documents\uOttawa\Second Term\ELG7186 -  AI for Cyber Security\4- Project\elg7186-project-group_project_-3\data\raw"
AnswersPATH = os.path.join(RootPath,'answers')
DataSetPath = r"D:\Hagar\Documents\uOttawa\Second Term\ELG7186 -  AI for Cyber Security\4- Project\elg7186-project-group_project_-3\data\raw"

In [4]:
DATASET_VER = 5.2
insiders_df = pd.read_csv(os.path.join(AnswersPATH,"insiders.csv"))
insiders_df = insiders_df[(insiders_df['dataset'] == DATASET_VER)]
insiders_df['start'] = pd.to_datetime(insiders_df['start'])
insiders_df = insiders_df.sort_values(by='start', ascending=True)
insiders_df.head()

,dataset,scenario,details,user,start,end
177,5.2,4,r5.2-4-OHS0036.csv,OHS0036,2010-04-16 12:32:57,07/09/2010 10:32:40
148,5.2,3,r5.2-3-MPF0690.csv,MPF0690,2010-06-17 10:00:38,06/18/2010 20:22:06
111,5.2,2,r5.2-2-VCF1602.csv,VCF1602,2010-06-23 15:32:14,08/11/2010 18:01:01
128,5.2,2,r5.2-2-CKP0630.csv,CKP0630,2010-06-28 08:13:30,08/13/2010 12:59:03
115,5.2,2,r5.2-2-ZIE0741.csv,ZIE0741,2010-06-29 13:16:09,08/27/2010 18:13:31


In [5]:
# get user functional dictionary
all_files = glob.glob(os.path.join(DataSetPath,"LDAP/*.csv"))
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
LDAP_df = pd.concat(li, axis=0, ignore_index=True).drop_duplicates()

print(f'length of LDAP : {len(LDAP_df)}')
LDAP_df.head(10)

length of LDAP : 3581


,employee_name,user_id,email,role,projects,business_unit,functional_unit,department,team,supervisor
0,Maisie Maggy Kline,MMK1532,Maisie.Maggy.Kline@dtaa.com,Technician,NaN,2 - Executive,5 - SalesAndMarketing,3 - FieldService,3 - RegionalFieldService,Yvette Naomi Cortez
1,Nicole Tanya Burt,NTB0710,Nicole.Tanya.Burt@dtaa.com,SoftwareQualityEngineer,Project 2,2 - Executive,2 - ResearchAndEngineering,3 - SoftwareManagement,1 - SQA,Brennan Thaddeus Dominguez
2,Madeline Tatum Donaldson,MTD0971,Madeline.Tatum.Donaldson@dtaa.com,ElectricalEngineer,NaN,1 - Executive,2 - ResearchAndEngineering,3 - SoftwareManagement,6 - EmbeddedSoftware,Todd Rajah Peterson
3,Nicholas Hamish Browning,NHB1529,Nicholas.Hamish.Browning@dtaa.com,Technician,NaN,2 - Executive,5 - SalesAndMarketing,3 - FieldService,3 - RegionalFieldService,Yvette Naomi Cortez
4,Hoyt Berk Wiley,HBW0057,Hoyt.Berk.Wiley@dtaa.com,ProjectManager,NaN,1 - Executive,2 - ResearchAndEngineering,1 - ProjectManagement,NaN,Rina Adena Horn
5,Patience Lesley Mccoy,PLM1716,Patience.Lesley.Mccoy@dtaa.com,Salesman,NaN,2 - Executive,5 - SalesAndMarketing,2 - Sales,4 - RegionalSales,Ethan Branden Garrett
6,Ulric Kirk Guthrie,UKG0386,Ulric.Kirk.Guthrie@dtaa.com,ProductionLineWorker,NaN,2 - Executive,3 - Manufacturing,3 - Assembly,1 - AssemblyDept,Prescott Jasper Collins
7,Guinevere Lisandra Chang,GLC1055,Guinevere.Lisandra.Chang@dtaa.com,ProductionLineWorker,NaN,1 - Executive,3 - Manufacturing,3 - Assembly,4 - AssemblyDept,Evangeline Yoko Malone
8,Ulric Ciaran Dunlap,UCD0608,Ulric.Ciaran.Dunlap@dtaa.com,StockroomClerk,NaN,1 - Executive,3 - Manufacturing,3 - Assembly,1 - Stockroom,Sydney Genevieve Nichols
9,Coby Baxter Nguyen,CBN1983,Coby.Baxter.Nguyen@dtaa.com,LabManager,NaN,1 - Executive,2 - ResearchAndEngineering,2 - Research,6 - Lab,Dominique Karly Charles


In [6]:
from datetime import timedelta

def is_weekend(x):
    if (x == 5) or (x==6):
        return "weekend"
    else:
        return "weekday"

def is_working_hour(x):
    if(x >= 7 and x <= 18):
        return 'working_hour'
    else:
        return "after_hours"
    
def get_user_pc(x, unique_pcs):
    '''
    '''
    user = unique_pcs[unique_pcs['pc'] == x['pc']].user.values
    if(len(user)):
        return user[0]

def get_supervisor(user_id, ldap_file=LDAP_df):
    supervisor_name = ldap_file[ldap_file['user_id'] == user_id].supervisor.values[0]
    supervisor_id = ldap_file[ldap_file['employee_name'] == supervisor_name].user_id
    if(len(supervisor_id)):
        return supervisor_id.values[0]
    else:
        return None

def is_personal_pc(x, user_pcs_df):
    '''
    '''
    user_pc = user_pcs_df[user_pcs_df['user'] == x.user].pc.values[0]
    
    if(user_pc == x.pc):
        return "personal_pc"
    
    supervisor = get_supervisor(x.user)
    if(supervisor is not None):
        supervisor_pc = user_pcs_df[user_pcs_df['user'] == supervisor].pc.values[0]
        if(supervisor_pc == x.pc):
            return "supervisor_pc"
        else:
            return "other_pc"
    
    return "other_pc"
    
def map_user_pcs(df, time_period=15):
    '''
    '''
    user_pc_df = pd.DataFrame(columns=['pc', 'user'])
    start_date = df.date.min()
    end_date =  start_date + timedelta(days=time_period)
    
    time_period_logons = df[(df['date'] > start_date) & (df['date'] < end_date)]

    user_pc_df['pc'] = time_period_logons['pc'].unique()
    user_pc_counts = time_period_logons.groupby(['user', 'pc']).size()
    user_pc_counts = user_pc_counts.to_frame(name = 'count').reset_index()
    idxs = user_pc_counts.groupby(['user'])['count'].transform(max) == user_pc_counts['count']
    unique_pcs = user_pc_counts[idxs]
    user_pc_df['user'] = user_pc_df.apply(get_user_pc, args=[unique_pcs], axis=1)
    
    return user_pc_df.dropna()

In [7]:
all_logon_df = pd.read_csv(r"D:\Hagar\Documents\uOttawa\Second Term\ELG7186 -  AI for Cyber Security\4- Project\Dataset\r5.2\logon.csv")
all_logon_df['date'] = pd.to_datetime(all_logon_df['date'])
all_logon_df.head()

,id,date,user,pc,activity
0,{Q4D5-W4HH44UC-5188LWZK},2010-01-02 02:24:51,JBI1134,PC-0168,Logon
1,{G7V0-S4TP95SA-9203AOGR},2010-01-02 02:38:28,JBI1134,PC-0168,Logoff
2,{B4U7-K4DB84LM-2657VBFY},2010-01-02 04:55:52,JBI1134,PC-2320,Logon
3,{K2E1-W7VG04OA-3686THSV},2010-01-02 05:02:28,JBI1134,PC-2320,Logoff
4,{Z7B2-Z8AS75YE-4485NAFE},2010-01-02 06:35:00,HMI1448,PC-9352,Logon


In [8]:
user_pcs_df = map_user_pcs(all_logon_df)
user_pcs_df

,pc,user
0,PC-0168,KKP0064
1,PC-2320,ADR1156
2,PC-9352,HMI1448
3,PC-5439,JEH0587
4,PC-2683,KMC1934
...,...,...
1997,PC-3634,FWB1999
1998,PC-3877,IRN1736
1999,PC-4325,MSK0117
2000,PC-0845,PBS0120


## Email Features

In [9]:
def internal_recepients_count(to):
    recepients = to.split(";")
    internal_recepients = [rec for rec in recepients if "dtaa" in rec]
    return len(internal_recepients)

def is_internal_email(to):
    recepients = to.split(";")
    internal_recepients = [rec for rec in recepients if "dtaa" in rec]
    if len(internal_recepients) == len(recepients):
        return 1
    else: 
        return 0
    
def unique_recepients(to):
    daily_recepients = ";".join(to)
    daily_recepients = daily_recepients.split(";")
    return np.unique(np.array(daily_recepients)).shape[0]

In [10]:
user_daily_emails = pd.read_csv(os.path.join(DataSetPath,'email.csv'))
user_daily_emails['date'] = pd.to_datetime(user_daily_emails['date'])
user_daily_emails["day"] = user_daily_emails["date"].dt.normalize()
user_daily_emails["hour"] = user_daily_emails["date"].dt.hour

user_daily_emails["is_working_hour"] = user_daily_emails['hour'].apply(is_working_hour)
user_daily_emails["is_personal_pc"] = user_daily_emails.apply(is_personal_pc, args=[user_pcs_df], axis=1)
user_daily_emails.head()

,id,date,user,pc,to,cc,bcc,from,activity,size,attachments,content,day,hour,is_working_hour,is_personal_pc
0,{Y4F7-N0DM86BL-9295AJOV},2010-01-02 07:47:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,5353488,13.doc(1835701);army.doc(1931135);covered.jpg(...,remind sought suffered onto conference establi...,2010-01-02,7,working_hour,personal_pc
1,{O6W5-S9OE61YZ-9954AAJV},2010-01-02 07:48:39,GVM1337,PC-6338,Paki.Kermit.Fox@dtaa.com;Xenos.Nicolas.Church@...,Alan.Flynn.Pope@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Receive,5353488,13.doc(1835701);army.doc(1931135);covered.jpg(...,remind sought suffered onto conference establi...,2010-01-02,7,working_hour,personal_pc
2,{A2G6-T7FE08CK-7601IYDT},2010-01-02 07:50:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Send,22721,NaN,warned repair devastated twin passing french h...,2010-01-02,7,working_hour,personal_pc
3,{V9E1-E4KG05KT-8603JHAW},2010-01-02 07:50:40,GVM1337,PC-6338,Lara.Cathleen.Rich@dtaa.com,NaN,Gregory.Victor.Mccarty@dtaa.com,Gregory.Victor.Mccarty@dtaa.com,Send,86054,NaN,repulsed francis everything three rationalised...,2010-01-02,7,working_hour,personal_pc
4,{W8E0-S2GA51NB-8855DBRL},2010-01-02 07:51:20,GVM1337,PC-6338,Guinevere_A_Taylor@cox.net,GVM9137@charter.net,NaN,GVM9137@charter.net,Receive,22721,NaN,warned repair devastated twin passing french h...,2010-01-02,7,working_hour,personal_pc


In [11]:
user_daily_emails['attachments'] = user_daily_emails['attachments'].replace(np.nan,'',regex=True)

user_daily_emails["working_hour"] = user_daily_emails["is_working_hour"].apply(lambda x: 1 if x == "working_hour" else 0)
user_daily_emails["after_hours"] = user_daily_emails["is_working_hour"].apply(lambda x: 1 if x == "after_hours" else 0)

user_daily_emails["user_pc"] = user_daily_emails["is_personal_pc"].apply(lambda x: 1 if x == "personal_pc" else 0)
user_daily_emails["supervisor_pc"] = user_daily_emails["is_personal_pc"].apply(lambda x: 1 if x == "supervisor_pc" else 0)
user_daily_emails["other_pc"] = user_daily_emails["is_personal_pc"].apply(lambda x: 1 if x == "other_pc" else 0)
user_daily_emails["other_pcs"] = user_daily_emails["supervisor_pc"] + user_daily_emails["other_pc"]

user_daily_emails["has_attachments"] = user_daily_emails["attachments"].apply(lambda x: 1 if len(x) > 0 else 0)
user_daily_emails["internal_recepients"] = user_daily_emails["to"].apply(internal_recepients_count)

user_daily_emails["is_internal_email"] = user_daily_emails["to"].apply(is_internal_email)
user_daily_emails["from_internal_email"] = user_daily_emails["from"].apply(lambda x: 1 if "dtaa" in x else 0)
user_daily_emails["from_external_email"] = user_daily_emails["from"].apply(lambda x: 1 if "dtaa" not in x else 0)

In [12]:
total_daily_recepients = user_daily_emails[user_daily_emails.activity == "Send"].groupby(['day', 'user'])["to"].apply(list).to_frame().reset_index()

In [13]:
sent_daily_emails = user_daily_emails[user_daily_emails.activity == "Send"].groupby(['day', 'user']).agg(
    sent_count = ("id", "count"),
    sent_internal_emails_count = ("is_internal_email", sum),
    sent_internal_recepients_count = ("internal_recepients", sum),
    sent_after_hour_count = ("after_hours", sum),
    sent_with_attachments_count = ("has_attachments", sum),
    sent_from_other_pcs_count = ("other_pcs", sum)    
).reset_index()

In [14]:
sent_daily_emails["distinct_recepients_count"] = total_daily_recepients["to"].apply(unique_recepients)

In [16]:
sent_daily_emails.to_csv("extracted_features/sent_daily_emails.csv", index=False)

In [17]:
received_daily_emails = user_daily_emails[user_daily_emails.activity == "Receive"].groupby(['day', 'user']).agg(
    received_count = ("id", "count"),
    received_internal_emails_count = ("from_internal_email", sum),
    received_external_emails_count = ("from_external_email", sum),
    recieved_after_hour_count = ("after_hours", sum),
    receive_with_attachments_count = ("has_attachments", sum),
    received_from_other_pcs_count = ("other_pcs", sum)    
).reset_index()

In [18]:
received_daily_emails.to_csv("extracted_features/received_daily_emails.csv", index=False)

## Device Features

In [19]:
user_daily_devices = pd.read_csv(os.path.join(DataSetPath,'device.csv'))
user_daily_devices['date'] = pd.to_datetime(user_daily_devices['date'])
user_daily_devices["day"] = user_daily_devices["date"].dt.normalize()
user_daily_devices["hour"] = user_daily_devices["date"].dt.hour

user_daily_devices["is_working_hour"] = user_daily_devices['hour'].apply(is_working_hour)
user_daily_devices["is_personal_pc"] = user_daily_devices.apply(is_personal_pc, args=[user_pcs_df], axis=1)
user_daily_devices.head()

,id,date,user,pc,file_tree,activity,day,hour,is_working_hour,is_personal_pc
0,{I3E7-Z8KN26QC-0847RGRG},2010-01-02 08:42:18,ESM1828,PC-7721,R:\;R:\ESM1828;R:\584Xlk8,Connect,2010-01-02,8,working_hour,personal_pc
1,{Q0Z2-I6OI75TP-7558HMVN},2010-01-02 08:44:11,ESM1828,PC-7721,NaN,Disconnect,2010-01-02,8,working_hour,personal_pc
2,{E8D8-X5NB52VQ-5232FIDW},2010-01-02 19:13:36,VRP0267,PC-8461,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect,2010-01-02,19,after_hours,other_pc
3,{B1E2-Z3MS26GZ-5290QMFQ},2010-01-02 19:15:49,VRP0267,PC-8461,NaN,Disconnect,2010-01-02,19,after_hours,other_pc
4,{P7V4-W2QB19PV-4652XYPX},2010-01-02 22:02:22,VRP0267,PC-4916,R:\;R:\VRP0267;R:\45L8257;R:\61x91b6,Connect,2010-01-02,22,after_hours,other_pc


In [20]:
user_daily_devices["after_hours"] = user_daily_devices["is_working_hour"].apply(lambda x: 1 if x == "after_hours" else 0)

user_daily_devices["user_pc"] = user_daily_devices["is_personal_pc"].apply(lambda x: 1 if x == "personal_pc" else 0)
user_daily_devices["supervisor_pc"] = user_daily_devices["is_personal_pc"].apply(lambda x: 1 if x == "supervisor_pc" else 0)
user_daily_devices["other_pc"] = user_daily_devices["is_personal_pc"].apply(lambda x: 1 if x == "other_pc" else 0)
user_daily_devices["other_pcs"] = user_daily_devices["supervisor_pc"] + user_daily_devices["other_pc"]

In [21]:
daily_devices = user_daily_devices[user_daily_devices.activity == "Connect"].groupby(['day', 'user']).agg(
    device_access_count = ("id", "count"),
    device_total_pc_count = ("pc", "nunique"),
    after_hour_device_access_count = ("after_hours", sum),
    user_pc_device_access_count = ("user_pc", sum),
    other_pc_device_access_count = ("other_pcs", sum)
).reset_index()

In [22]:
daily_devices.to_csv("extracted_features/daily_devices.csv", index=False)

## File Features

In [23]:
user_daily_files = pd.read_csv(os.path.join(DataSetPath, "file.csv"))
user_daily_files['date'] = pd.to_datetime(user_daily_files['date'])
user_daily_files["day"] = user_daily_files["date"].dt.normalize()
user_daily_files["hour"] = user_daily_files["date"].dt.hour

user_daily_files["is_working_hour"] = user_daily_files['hour'].apply(is_working_hour)
user_daily_files["is_personal_pc"] = user_daily_files.apply(is_personal_pc, args=[user_pcs_df], axis=1)
user_daily_files.head()

,id,date,user,pc,filename,activity,to_removable_media,from_removable_media,content,day,hour,is_working_hour,is_personal_pc
0,{G5X3-Z3TT87MZ-1125ZWHW},2010-01-02 19:14:34,VRP0267,PC-8461,R:\ICA5T4Q8.doc,File Open,False,True,D0-CF-11-E0-A1-B1-1A-E1 devised mine officiall...,2010-01-02,19,after_hours,other_pc
1,{U5N0-D9KT82SH-1353KTHL},2010-01-02 19:15:01,VRP0267,PC-8461,C:\GPH0932\NFR12BZA.doc,File Open,False,False,D0-CF-11-E0-A1-B1-1A-E1 islands consistently o...,2010-01-02,19,after_hours,other_pc
2,{J8K3-E7NP02GP-8515JPPH},2010-01-02 23:06:10,VRP0267,PC-1985,R:\45L8257\XOCNOM4O.pdf,File Write,True,False,25-50-44-46-2D greater he did respectable peop...,2010-01-02,23,after_hours,other_pc
3,{W5T9-V3BA42RO-8223ANRJ},2010-01-03 09:05:18,MAB1340,PC-1272,R:\29Z1tL4\F8DHK5K7.doc,File Open,False,True,D0-CF-11-E0-A1-B1-1A-E1 whether alternative in...,2010-01-03,9,working_hour,personal_pc
4,{Q0K4-O9CI90HJ-9787SZTW},2010-01-03 09:16:32,MAB1340,PC-1272,R:\29Z1tL4\F8DHK5K7.doc,File Write,True,False,D0-CF-11-E0-A1-B1-1A-E1 siding purpose met not...,2010-01-03,9,working_hour,personal_pc


In [25]:
user_daily_files["after_hours"] = user_daily_files["is_working_hour"].apply(lambda x: 1 if x == "after_hours" else 0)

user_daily_files["user_pc"] = user_daily_files["is_personal_pc"].apply(lambda x: 1 if x == "personal_pc" else 0)
user_daily_files["supervisor_pc"] = user_daily_files["is_personal_pc"].apply(lambda x: 1 if x == "supervisor_pc" else 0)
user_daily_files["other_pc"] = user_daily_files["is_personal_pc"].apply(lambda x: 1 if x == "other_pc" else 0)
user_daily_files["other_pcs"] = user_daily_files["supervisor_pc"] + user_daily_files["other_pc"]

In [26]:
daily_files = user_daily_files.groupby(['day', 'user']).agg(
    file_access_count = ("id", "count"),
    file_total_pc_count = ("pc", "nunique"),
    distinct_files_count = ("filename", "nunique"),
    after_hour_file_access_count = ("after_hours", sum),
    user_pc_file_access_count = ("user_pc", sum),
    other_pc_file_access_count = ("other_pcs", sum)
).reset_index()

In [27]:
daily_files.to_csv("extracted_features/daily_files.csv", index=False)

## HTTPs Features

In [28]:
user_daily_https = pd.read_csv(os.path.join(DataSetPath,"http.csv"))
user_daily_https['date'] = pd.to_datetime(user_daily_https['date'])
user_daily_https['day'] = pd.to_datetime(user_daily_https['date']).dt.normalize()
user_daily_https["hour"] = user_daily_https["date"].dt.hour

user_daily_https["is_working_hour"] = user_daily_https['hour'].apply(is_working_hour)
user_daily_https["is_personal_pc"] = user_daily_https.apply(is_personal_pc, args=[user_pcs_df], axis=1)
user_daily_https.head()

,id,date,user,pc,url,content,sld,day,hour,is_working_hour,is_personal_pc
0,{S1W7-N5RL66HS-8417FLPG},2010-01-02 07:28:57,VRP0267,PC-5687,http://shareasale.com/1962_South_Vietnamese_In...,build 1935 promontories william fled employees...,shareasale.com,2010-01-02,7,working_hour,personal_pc
1,{S3A7-O7EP60NA-9986PZVN},2010-01-02 07:37:20,IRH1224,PC-4126,http://linkedin.com/Harry_Chauvel/chetwode/ceb...,development wrong emotionally voters todays fl...,linkedin.com,2010-01-02,7,working_hour,personal_pc
2,{T9S7-R4CL46NO-2720HYVY},2010-01-02 07:37:40,VRP0267,PC-5687,http://inbox.com/Cheadle_Hulme/hulme/nanylfvfs...,told our excessive transport the prompted paid...,inbox.com,2010-01-02,7,working_hour,personal_pc
3,{Z0P6-H6QZ79BZ-3362JYNM},2010-01-02 07:39:45,VRP0267,PC-5687,http://weather.gov/Frank_Zappa/synclavier/onyn...,1915 although casualties done heading disabled...,weather.gov,2010-01-02,7,working_hour,personal_pc
4,{D1V2-A3YL76WV-6382SWTS},2010-01-02 07:42:08,IRH1224,PC-4126,http://cnn.com/Anton_Chekhov/taganrog/pbzcrafn...,114 early manston chosen series with that dyna...,cnn.com,2010-01-02,7,working_hour,personal_pc


In [29]:
user_daily_https["after_hours"] = user_daily_https["is_working_hour"].apply(lambda x: 1 if x == "after_hours" else 0)

user_daily_https["user_pc"] = user_daily_https["is_personal_pc"].apply(lambda x: 1 if x == "personal_pc" else 0)
user_daily_https["supervisor_pc"] = user_daily_https["is_personal_pc"].apply(lambda x: 1 if x == "supervisor_pc" else 0)
user_daily_https["other_pc"] = user_daily_https["is_personal_pc"].apply(lambda x: 1 if x == "other_pc" else 0)
user_daily_https["other_pcs"] = user_daily_https["supervisor_pc"] + user_daily_https["other_pc"]

In [30]:
daily_https = user_daily_https.groupby(['day', 'user']).agg(
    webvisits_count = ("id", "count"),
    http_total_pc_count = ("pc", "nunique"),
    distinct_url_count = ("sld", "nunique"),
    after_hour_webvisits_count = ("after_hours", sum)
).reset_index()

In [31]:
daily_https.to_csv("extracted_features/daily_https.csv", index=False)

## Logon Features

In [32]:
user_daily_logons = pd.read_csv(os.path.join(DataSetPath,'logon.csv'))
user_daily_logons['date'] = pd.to_datetime(user_daily_logons['date'])
user_daily_logons["day"] = user_daily_logons["date"].dt.normalize()
user_daily_logons["hour"] = user_daily_logons["date"].dt.hour

user_daily_logons["is_working_hour"] = user_daily_logons['hour'].apply(is_working_hour)
user_daily_logons["is_personal_pc"] = user_daily_logons.apply(is_personal_pc, args=[user_pcs_df], axis=1)
user_daily_logons.head()

,id,date,user,pc,activity,day,hour,is_working_hour,is_personal_pc
0,{U0U5-J2AX64NN-7430KKNP},2010-01-02 07:27:00,VRP0267,PC-5687,Logon,2010-01-02,7,working_hour,personal_pc
1,{B0G5-P2XR80RE-6009PRBI},2010-01-02 07:34:00,IRH1224,PC-4126,Logon,2010-01-02,7,working_hour,personal_pc
2,{R3Z9-U5YR87BS-2979BOUZ},2010-01-02 07:40:00,ELT1370,PC-1929,Logon,2010-01-02,7,working_hour,personal_pc
3,{X0C4-S3RJ20AY-4832PPRI},2010-01-02 07:41:00,JMM0613,PC-7127,Logon,2010-01-02,7,working_hour,personal_pc
4,{U3L5-D8ZD99IK-6715VWBI},2010-01-02 07:45:00,GVM1337,PC-6338,Logon,2010-01-02,7,working_hour,personal_pc


In [33]:
user_daily_logons["after_hours"] = user_daily_logons["is_working_hour"].apply(lambda x: 1 if x == "after_hours" else 0)

user_daily_logons["user_pc"] = user_daily_logons["is_personal_pc"].apply(lambda x: 1 if x == "personal_pc" else 0)
user_daily_logons["supervisor_pc"] = user_daily_logons["is_personal_pc"].apply(lambda x: 1 if x == "supervisor_pc" else 0)
user_daily_logons["other_pc"] = user_daily_logons["is_personal_pc"].apply(lambda x: 1 if x == "other_pc" else 0)
user_daily_logons["other_pcs"] = user_daily_logons["supervisor_pc"] + user_daily_logons["other_pc"]

In [34]:
daily_logons = user_daily_logons[user_daily_logons.activity == "Logon"].groupby(['day', 'user']).agg(
    logon_count = ("id", "count"),
    logon_total_pc_count = ("pc", "nunique"),
    after_hour_logon_count = ("after_hours", sum),
    user_pc_logon_count = ("user_pc", sum),
    other_pc_logon_count = ("other_pcs", sum)
).reset_index()

In [35]:
daily_logons.to_csv("extracted_features/daily_logons.csv", index=False)

## All Features

In [36]:
import glob 

feature_dfs = []
features_path = os.path.join(r"D:\Hagar\Documents\uOttawa\Second Term\ELG7186 -  AI for Cyber Security\4- Project\elg7186-project-group_project_-3\notebooks\extracted_features",'*.csv')
for path in glob.glob(features_path):
    df = pd.read_csv(path) 
    feature_dfs.append(df)

In [37]:
from functools import reduce

merged_features = reduce(lambda left, right: pd.merge(left, right, on=['day', 'user'], how='outer'), feature_dfs).fillna(0)
merged_features

,day,user,device_access_count,device_total_pc_count,after_hour_device_access_count,user_pc_device_access_count,other_pc_device_access_count,file_access_count,file_total_pc_count,distinct_files_count,...,recieved_after_hour_count,receive_with_attachments_count,received_from_other_pcs_count,sent_count,internal_emails_count,internal_recepients_count,sent_after_hour_count,sent_with_attachments_count,sent_from_other_pcs_count,distinct_recepients_count
0,2010-01-02,ESM1828,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,16.0,5.0,12.0,0.0,5.0,0.0,23.0
1,2010-01-02,VRP0267,4.0,4.0,4.0,0.0,4.0,3.0,2.0,3.0,...,0.0,3.0,0.0,10.0,8.0,13.0,0.0,3.0,0.0,16.0
2,2010-01-03,KSS1005,5.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,9.0,9.0,19.0,0.0,3.0,0.0,11.0
3,2010-01-03,MAB1340,2.0,1.0,0.0,2.0,0.0,4.0,1.0,2.0,...,0.0,7.0,0.0,10.0,5.0,7.0,0.0,3.0,0.0,12.0
4,2010-01-03,VCF1602,5.0,1.0,0.0,5.0,0.0,3.0,1.0,3.0,...,0.0,7.0,0.0,15.0,12.0,24.0,0.0,6.0,0.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62525,2011-04-22,JVM0772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62526,2011-04-22,YBD1929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62527,2011-05-14,CCP0774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62528,2011-05-14,PAA0121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
merged_features.to_csv("extracted_features/merged_features.csv", index=False)